In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spstats
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
train = pd.read_csv('train.csv',delimiter='|')
test = pd.read_csv('test.csv',delimiter='|')
whole_data = pd.concat([train[['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemVoids',
       'scansWithoutRegistration', 'quantityModifications',
       'scannedLineItemsPerSecond', 'valuePerSecond',
       'lineItemVoidsPerPosition']], test], axis=0)

In [ ]:
whole_data.columns

In [ ]:
whole_data.shape

In [ ]:
train.columns

In [ ]:
train[train['fraud']==0].describe()

In [ ]:
train[train['fraud']==1].describe()

### Feature Generation

In [ ]:
train['totalScanned'] = train['scannedLineItemsPerSecond'] * train['totalScanTimeInSeconds']
whole_data['totalScanned'] = whole_data['scannedLineItemsPerSecond'] * whole_data['totalScanTimeInSeconds']
# Extend with the additional features listed in the presentation.

In [ ]:
#train.columns

### Automatic feature generation with featuretools

https://www.kaggle.com/willkoehrsen/automated-feature-engineering-tutorial

In [ ]:
import featuretools as ft

In [ ]:
es = ft.EntitySet(id = 'customers')

In [ ]:
es = es.entity_from_dataframe(entity_id='customers',dataframe=whole_data,
                              variable_types = {'trustLevel': ft.variable_types.Categorical},index='id',make_index=True)

In [ ]:
es['customers']

In [ ]:
# List possible aggregation primitives 
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
#primitives[primitives['type'] == 'aggregation'].head(20)

In [ ]:
# List possible transformation primitives
#primitives[primitives['type'] == 'transform'].head(40)

In [ ]:

# Can take a while to compute!!!

features, feature_names = ft.dfs(entityset = es, target_entity = 'customers', 
                                 agg_primitives = ['skew','trend','median', 'mean', 'max', 'std'],
                                 trans_primitives = ['subtract', 'percentile'])

In [ ]:
features.head()

In [ ]:
features.columns

### Those Features now have to be validated or let's say a feature selection has to be performed SVD or PCA. Besides that, one can play with featuretools and change the selected transformation or aggregation primitives.

- needs to be done


____________________________________________________________________

### Evaluation

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier

df = pd.read_csv('train.csv', sep='|')
X, y = df.drop(columns='fraud'), df['fraud']

cv = StratifiedKFold(n_splits=10, random_state=42)
def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))
profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

sum(cross_validate(XGBClassifier(), X, y=y, cv=10, scoring=profit_scoring)['test_score'])

In [ ]:
X = train[['trustLevel', 'totalScanTimeInSeconds', 'grandTotal', 'lineItemVoids',
       'scansWithoutRegistration', 'quantityModifications',
       'scannedLineItemsPerSecond', 'valuePerSecond',
       'lineItemVoidsPerPosition']]

In [ ]:
y = train['fraud']

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs').fit(X, y)

In [ ]:
clf.score(X,y)